In [1]:
from pymongo import MongoClient
import pandas as pd
import seaborn as sns
import numpy as np
import re
import plotly.express as px
from plotly import graph_objects as go
import matplotlib.pyplot as plt

## Calling the service MongoDB

In [2]:
client = MongoClient("localhost:27017")

In [3]:
db = client["Ironhack"]

In [4]:
comp = db.get_collection("companies")